In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap
from shapely.geometry import MultiPoint

In [3]:
import os
import sys

from os.path import dirname
UTILS_PATH=os.environ.get('REPO') + "/notebooks/commons"
sys.path.append(dirname(UTILS_PATH))

from commons import check_args, gen_paths, coordinates_bounds

In [4]:
import math

def safe_pow(e):
    result = math.inf
    try:
        result = float(2**e)
        return result
    except OverflowError as error:
        return result
    

In [5]:
def load_posts(input_path):
    df = pd.read_parquet(input_path)
    return df

In [6]:
if __name__ == "__main__":
    city = "manizales"
    property_type = "casas"
    post_type = "venta"
    
    #city, property_type, post_type = check_args()
    base_path = os.environ.get('REPO')
    
    raw_path, clean_path = gen_paths(city, property_type, post_type)
    
    raw_path = base_path + raw_path
    clean_path = base_path + clean_path

In [7]:
MSG = """
Hay {} publicaciones de {} en total
"""

posts = load_posts(clean_path)
print(MSG.format(posts.shape[0], post_type))
posts.head()


Hay 1992 publicaciones de venta en total



,surface,rooms,baths,garages,price,latitude,description,longitude,location,url,admon,estrato,antiguedad,fid,city,property_type,post_type,price_m2
1,180.0,4.0,1.0,1.0,250.0,5.061220,"Casa ubicada en el barrio La Sultana, consta d...",-75.472801,Manizales LA SULTANA,https://www.fincaraiz.com.co/casa-en-venta/man...,0.0,3.0,16 a 30,5151631,manizales,casas,venta,1.388889
2,130.0,4.0,2.0,0.0,225.0,5.047913,"Area de 130 mtrs, excelente ubicacion, casa co...",-75.501884,Manizales MALHABAR,https://www.fincaraiz.com.co/casa-en-venta/man...,0.0,3.0,16 a 30,4715541,manizales,casas,venta,1.730769
3,407.0,9.0,6.0,5.0,1200.0,5.069806,"Excelente inversion, venta de propiedad horizo...",-75.530762,Manizales La francia,https://www.fincaraiz.com.co/casa-en-venta/man...,0.0,6.0,desconocido,3333614,manizales,casas,venta,2.948403
4,118.0,3.0,3.0,1.0,450.0,5.027654,Vista Inmobiliaria ofrece en venta linda casa ...,-75.481209,Manizales La Florida,https://www.fincaraiz.com.co/casa-en-venta/man...,285000.0,4.0,desconocido,5392228,manizales,casas,venta,3.813559
5,115.0,3.0,3.0,2.0,370.0,5.038804,Venta casa en conjunto cerrado en el sector de...,-75.487785,Manizales La Florida,https://www.fincaraiz.com.co/casa-en-venta/man...,220000.0,4.0,1 a 8,5613118,manizales,casas,venta,3.217391


## Mapa de calor precios

In [8]:
posts["price"].describe()

count    1992.000000
mean      470.901334
std       387.405646
min        60.000000
25%       240.000000
50%       350.000000
75%       600.000000
max      4000.000000
Name: price, dtype: float64

In [9]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

posts.loc[:, "price_exploded"] =  posts["price"].apply(safe_pow)

max_amount = posts['price_exploded'].max()

hm_wide = HeatMap( 
    posts[["latitude", "longitude", "price_exploded"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)

## Mapa de calor número de publicaciones

In [10]:
from sklearn.cluster import DBSCAN

In [11]:
coords = posts[['latitude','longitude']].values

kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian

min_samples = 20

In [12]:
## Geo clustering with examples
# https://stackoverflow.com/questions/24762435/clustering-geo-location-coordinates-lat-long-pairs-using-kmeans-algorithm-with
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [13]:
centroids = []
clusters_size = len(db.labels_)

for n in range(clusters_size):
    cluster = posts[['latitude','longitude']][db.labels_ == n].copy()
    if len(cluster) > 0:
        cluster.loc[:, "cluster_id"] = n
        
        multi_point = MultiPoint(cluster.values)
        cluster.loc[:, "cluster_latitude"] = multi_point.centroid.x
        cluster.loc[:, "cluster_longitude"] = multi_point.centroid.y
        
    centroids.append(cluster)
        
clusters_df = pd.concat(centroids)
clusters_df = clusters_df.reset_index(drop=True)
clusters_df.head()

,latitude,longitude,cluster_id,cluster_latitude,cluster_longitude
0,5.070896,-75.513817,0.0,5.070322,-75.513809
1,5.070290,-75.513878,0.0,5.070322,-75.513809
2,5.070275,-75.513817,0.0,5.070322,-75.513809
3,5.070232,-75.513733,0.0,5.070322,-75.513809
4,5.070275,-75.513817,0.0,5.070322,-75.513809


In [14]:
clusters_coordinates = clusters_df[["cluster_id", "latitude", "longitude"]].groupby(["latitude", "longitude"]).count()

coordinates = list(zip(*clusters_coordinates.index))
clusters_coordinates.loc[:, "latitude"] = coordinates[0]
clusters_coordinates.loc[:, "longitude"] = coordinates[1]

clusters_coordinates.loc[:, "count"] = clusters_coordinates["cluster_id"].apply(safe_pow)
clusters_coordinates = clusters_coordinates.reset_index(drop=True)

clusters_coordinates.head()

,cluster_id,latitude,longitude,count
0,1,5.029472,-75.457619,2.0
1,1,5.029472,-75.457130,2.0
2,21,5.029500,-75.457329,2097152.0
3,1,5.029666,-75.456413,2.0
4,1,5.029881,-75.458328,2.0


In [15]:
folium_hmap = folium.Map(location=[
    np.mean([coordinates_bounds[city]["lat"]["lower"], coordinates_bounds[city]["lat"]["upper"]]),
    np.mean([coordinates_bounds[city]["lon"]["lower"], coordinates_bounds[city]["lon"]["upper"]])
], zoom_start=13, tiles="OpenStreetMap")

max_amount = clusters_coordinates['count'].max()

hm_wide = HeatMap( 
    clusters_coordinates[["latitude", "longitude", "count"]],
    min_opacity=0.2,
    max_val=max_amount,
    radius=7, blur=1, 
    max_zoom=15, 
    gradient={.1:'yellow', .5: 'orange',  1: 'red'}
)

folium_hmap.add_child(hm_wide)